2-1. Nate 뉴스기사 제목 스크래핑하기

https://news.nate.com/recent?mid=n0100
최신뉴스, 정치 , 경제, 사회, 세계, IT/과학
6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑하기

Image, 기사제목, 기사링크

뉴스기사의 Image를 출력하세요.

1) Image의 도메인이름이 포함된 경로와 src 속성의 경로를 합치려면 urljoin 함수를 사용하세요.

In [ ]:
from urllib.parse import urljoin
url = 'https://news.nate.com/recent?mid=n0100'
src='//thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408335_high.jpg'

2) Image 출력은 Image 클래스와 display 함수를 사용하세요.


In [ ]:
from IPython.display import Image, display

3) img 엘리먼트의 존재 여부를 체크하신 후에 src 속성의 이미지를 경로를 추출하기
=> Image가 없는 뉴스도 있기 때문에

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

def scrape_nate_news(section_name, url):
    """
    Scrapes news articles from a given Nate News section URL,
    extracts image, title, and link, and displays the image.

    Args:
        section_name (str): The name of the news section (e.g., "최신뉴스").
        url (str): The URL of the news section.
    """
    print(f"--- {section_name} ---")
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all list items containing news articles
        articles = soup.select('.newsContents .clear') # Or other appropriate selector if needed

        if not articles:
            print(f"No articles found for {section_name}. Check the CSS selector.")
            return

        for article in articles:
            # Extract image
            img_tag = article.select_one('img')
            if img_tag and img_tag.has_attr('src'):
                img_src = img_tag['src']
                # Construct full image URL using urljoin
                full_img_url = urljoin(url, img_src)
                print(f"Image URL: {full_img_url}")
                try:
                    display(Image(url=full_img_url))
                except Exception as e:
                    print(f"Could not display image from {full_img_url}: {e}")
            else:
                print("No image found for this article.")

            # Extract title and link
            title_tag = article.select_one('.tb .tit') # Assuming the title is within a .tb element and has a .tit class
            if title_tag:
                link_tag = title_tag.select_one('a')
                if link_tag and link_tag.has_attr('href'):
                    title = link_tag.get_text(strip=True)
                    link = urljoin(url, link_tag['href']) # Use urljoin for relative links
                    print(f"Title: {title}")
                    print(f"Link: {link}\n")
                else:
                    print("No link found for this title.")
            else:
                print("No title found for this article.\n")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {section_name} from {url}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred for {section_name}: {e}")

# URLs for the different Nate News sections
nate_news_sections = {
    "최신뉴스": "https://news.nate.com/recent?mid=n0100",
    "정치": "https://news.nate.com/politics?mid=n1001",
    "경제": "https://news.nate.com/economy?mid=n1002",
    "사회": "https://news.nate.com/society?mid=n1003",
    "세계": "https://news.nate.com/world?mid=n1004",
    "IT/과학": "https://news.nate.com/it?mid=n1005"
}

# Scrape each section
for section_name, url in nate_news_sections.items():
    scrape_nate_news(section_name, url)

--- 최신뉴스 ---
No articles found for 최신뉴스. Check the CSS selector.
--- 정치 ---
No articles found for 정치. Check the CSS selector.
--- 경제 ---
No articles found for 경제. Check the CSS selector.
--- 사회 ---
No articles found for 사회. Check the CSS selector.
--- 세계 ---
No articles found for 세계. Check the CSS selector.
--- IT/과학 ---
No articles found for IT/과학. Check the CSS selector.


2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기
:  하나의 웹툰의 제목(title)과 회차번호(no),회차의URL(url) 을 입력으로 받는 함수를 선언합니다.

In [ ]:
def download_one_episode(title,no,url):

아래와 같이 호출합니다.

In [ ]:
download_one_episode('일렉시드',341,'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')


img\일렉시드\341 디렉토리가 생성되며, 그 디렉토리 아래에 웹툰 image들이 다운로드 되도록 해주세요.

In [2]:
import os
import requests
from bs4 import BeautifulSoup

def download_one_episode(title, no, url):
    """
    네이버 웹툰의 특정 회차 이미지를 다운로드합니다.

    Args:
        title (str): 웹툰의 제목.
        no (int): 회차 번호.
        url (str): 해당 회차의 URL.
    """
    # 다운로드 경로 설정: img/웹툰제목/회차번호
    download_dir = os.path.join("img", title, str(no))
    os.makedirs(download_dir, exist_ok=True)
    print(f"다운로드 경로: {download_dir}")

    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 발생
    except requests.exceptions.RequestException as e:
        print(f"URL 요청 중 오류 발생: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    # 웹툰 이미지를 포함하는 div 태그 찾기 (클래스 이름이 'wt_viewer'인 div)
    viewer_div = soup.find('div', class_='wt_viewer')

    if not viewer_div:
        print("웹툰 이미지를 찾을 수 없습니다. HTML 구조가 변경되었을 수 있습니다.")
        return

    # viewer_div 내의 모든 이미지 태그 찾기
    images = viewer_div.find_all('img')

    if not images:
        print("다운로드할 이미지가 없습니다.")
        return

    for i, img in enumerate(images):
        img_url = img.get('src')
        if img_url and img_url.startswith(('http://', 'https://')):
            try:
                img_data = requests.get(img_url).content
                img_name = f"{i+1:03d}.jpg"  # 이미지 파일명 (예: 001.jpg, 002.jpg)
                img_path = os.path.join(download_dir, img_name)
                with open(img_path, 'wb') as f:
                    f.write(img_data)
                print(f"이미지 다운로드 완료: {img_name}")
            except requests.exceptions.RequestException as e:
                print(f"이미지 다운로드 중 오류 발생 ({img_url}): {e}")
        else:
            print(f"유효하지 않은 이미지 URL: {img_url}")

# 함수 호출 예시
download_one_episode('일렉시드', 341, 'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')

다운로드 경로: img\일렉시드\341
이미지 다운로드 완료: 001.jpg
이미지 다운로드 완료: 002.jpg
이미지 다운로드 완료: 003.jpg
이미지 다운로드 완료: 004.jpg
이미지 다운로드 완료: 005.jpg
이미지 다운로드 완료: 006.jpg
이미지 다운로드 완료: 007.jpg
이미지 다운로드 완료: 008.jpg
이미지 다운로드 완료: 009.jpg
이미지 다운로드 완료: 010.jpg
이미지 다운로드 완료: 011.jpg
이미지 다운로드 완료: 012.jpg
이미지 다운로드 완료: 013.jpg
이미지 다운로드 완료: 014.jpg
이미지 다운로드 완료: 015.jpg
이미지 다운로드 완료: 016.jpg
이미지 다운로드 완료: 017.jpg
이미지 다운로드 완료: 018.jpg
이미지 다운로드 완료: 019.jpg
이미지 다운로드 완료: 020.jpg
이미지 다운로드 완료: 021.jpg
이미지 다운로드 완료: 022.jpg
이미지 다운로드 완료: 023.jpg
이미지 다운로드 완료: 024.jpg
이미지 다운로드 완료: 025.jpg
이미지 다운로드 완료: 026.jpg
이미지 다운로드 완료: 027.jpg
이미지 다운로드 완료: 028.jpg
이미지 다운로드 완료: 029.jpg
이미지 다운로드 완료: 030.jpg
이미지 다운로드 완료: 031.jpg
이미지 다운로드 완료: 032.jpg
이미지 다운로드 완료: 033.jpg
이미지 다운로드 완료: 034.jpg
이미지 다운로드 완료: 035.jpg
이미지 다운로드 완료: 036.jpg
이미지 다운로드 완료: 037.jpg
이미지 다운로드 완료: 038.jpg
이미지 다운로드 완료: 039.jpg
이미지 다운로드 완료: 040.jpg
이미지 다운로드 완료: 041.jpg
이미지 다운로드 완료: 042.jpg
이미지 다운로드 완료: 043.jpg
이미지 다운로드 완료: 044.jpg
이미지 다운로드 완료: 045.jpg
이미지 다운로드 완료: 046.jpg
이미지 다운로드 완료: